In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.layers import Dropout
import matplotlib.pyplot as plt

In [ ]:
!wget -O admin.csv 'https://raw.githubusercontent.com/nehanawar025/Cyberbulling_text_classification_using_NLP/main/admin.csv'

In [6]:
df = pd.read_csv('admin.csv')
df.drop('Comments', axis=1, inplace=True)

In [7]:
df.head(7)

,id,text,label
0,7789,@AAlwuhaib1977 Muslim mob violence against Hin...,racism
1,7790,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,normal
2,7791,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism
3,7792,"Finally I'm all caught up, and that sudden dea...",normal
4,7793,@carolinesinders @herecomesfran *hugs*,normal
5,7794,"Please, PLEASE start using ""is your discernmen...",normal
6,7795,@aymannathem As soon as ISIS chased all the mi...,racism


In [9]:
max_words = 1000  # Max number of words to consider as features
max_len = 100  # Max length of a sequence

In [10]:
tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df['text'].values)
sequences = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(sequences, maxlen=max_len)

In [11]:
# labels to categorical
label_dict = {'racism': 0, 'sexism': 1, 'toxic': 2, 'normal': 3}
df['label'] = df['label'].map(label_dict)
y = to_categorical(df['label'].values)

In [12]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [70]:
# LSTM
emb_dim = 128


model = Sequential()
model.add(Embedding(max_words, emb_dim, input_length=max_len))
model.add(SpatialDropout1D(0.6))
model.add(LSTM(32, dropout=0.6, recurrent_dropout=0.6))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.6))
model.add(Dense(4, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
epochs = 20
batch_size = 32

result = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1,callbacks=[early_stopping])


Epoch 1/20
25/25 [==============================] - 10s 224ms/step - loss: 1.6807 - accuracy: 0.2550 - val_loss: 1.6404 - val_accuracy: 0.3750
Epoch 2/20
25/25 [==============================] - 4s 148ms/step - loss: 1.6109 - accuracy: 0.2977 - val_loss: 1.5788 - val_accuracy: 0.4300
Epoch 3/20
25/25 [==============================] - 4s 148ms/step - loss: 1.5552 - accuracy: 0.3329 - val_loss: 1.5253 - val_accuracy: 0.4450
Epoch 4/20
25/25 [==============================] - 5s 185ms/step - loss: 1.4923 - accuracy: 0.4058 - val_loss: 1.4618 - val_accuracy: 0.4100
Epoch 5/20
25/25 [==============================] - 4s 170ms/step - loss: 1.4219 - accuracy: 0.4133 - val_loss: 1.3745 - val_accuracy: 0.4500
Epoch 6/20
25/25 [==============================] - 4s 148ms/step - loss: 1.3182 - accuracy: 0.4648 - val_loss: 1.2746 - val_accuracy: 0.5400
Epoch 7/20
25/25 [==============================] - 4s 151ms/step - loss: 1.2191 - accuracy: 0.5151 - val_loss: 1.1835 - val_accuracy: 0.5000
Epoch